# Mlflow to BentoML

[BentoML](http://bentoml.ai) is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps**.

[MLflow](https://mlflow.org/) is an open source platform for the machine learning lifecycle, including experimentation, reproducibility, deployment, and a central model registry.

You might want to use Mlflow to keep track of your  training but you would prefer to use BentoML to deploy your models in productions. You can see a comparison between the two [here](https://docs.bentoml.org/en/latest/faq.html?highlight=mlflow#how-does-bentoml-compare-to-mlflow).

This notebook will demonstrate you how you can load a model from Mlflow model and package it with BentoML for deployment. We will break it down in the following parts:
1. Train a model based on iris dataset and save it using MLflow
2. Load the model from MLflow and package it with BentoML
3. Containerize the model with docker

BentoML requires python 3.6 or above, install dependencies via `pip`:

In [1]:
# Install PyPI packages required in this guide, including BentoML
!pip install -q 'bentoml' 'scikit-learn>=0.23.2' 'mlflow>=1.13.1' 'matplotlib'

## 1. Train a model and save it using MLflow
Like in the quick-start, let's train a classifier model on the [Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set):

In [4]:
from sklearn import svm
from sklearn import datasets
import mlflow

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training and saving in MLflow
clf = svm.SVC(gamma='scale')
with mlflow.start_run() as run:
    clf.fit(X, y)
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="model",
        signature=mlflow.models.signature.infer_signature(X),
    )

The model has been trained and saved in Mlflow. You can see it using the mlflow ui by running 

In [5]:
# We use a different port than bentoml default service to avoid conflicts
!mlflow ui --port=5001

[2021-02-14 21:34:12 +0100] [25225] [INFO] Starting gunicorn 20.0.4
[2021-02-14 21:34:12 +0100] [25225] [INFO] Listening at: http://127.0.0.1:5001 (25225)
[2021-02-14 21:34:12 +0100] [25225] [INFO] Using worker: sync
[2021-02-14 21:34:12 +0100] [25227] [INFO] Booting worker with pid: 25227
^C
[2021-02-14 21:34:14 +0100] [25225] [INFO] Handling signal: int
[2021-02-14 21:34:14 +0100] [25227] [INFO] Worker exiting (pid: 25227)


# 2. Load the model from MLflow and package it with BentoML

Like in the quick-start, the first step is creating a
prediction service class, which defines the models required and the inference APIs which
contains the serving logic. Here is a minimal prediction service created for serving
the iris classifier model trained above:

In [6]:
%%writefile iris_classifier.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):
    """
    A minimum prediction service exposing a Scikit-learn model
    """

    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        An inference API named `predict` with Dataframe input adapter, which codifies
        how HTTP requests or CSV files are converted to a pandas Dataframe object as the
        inference API function input
        """
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


This code defines a prediction service that packages a scikit-learn model and provides
an inference API that expects a `pandas.Dataframe` object as its input. 

We will now load this MLflow model. 

In [7]:
model_uri = f"runs:/{run.info.run_id}/model"
print(f"Retrieving model with uri={model_uri}")
mlflow_loaded_model = mlflow.sklearn.load_model(model_uri)

Retrieving model with uri=runs:/227b660c7c1c496aa14918758eb401c9/model


The following code packages the model loaded from MLflow with the prediction service class
`IrisClassifier` defined above, and then saves the IrisClassifier instance to disk 
in the BentoML format for distribution and deployment:

In [8]:
# import the IrisClassifier class defined above
from iris_classifier import IrisClassifier

# Create a iris classifier service instance
iris_classifier_service = IrisClassifier()

# Pack the newly trained model artifact
iris_classifier_service.pack('model', mlflow_loaded_model)

# Save the prediction service to disk for model serving
saved_path = iris_classifier_service.save()

[2021-02-14 21:34:46,624] INFO - BentoService bundle 'IrisClassifier:20210214213445_DB9735' saved to: /home/theodore/bentoml/repository/IrisClassifier/20210214213445_DB9735


The MLflow model has been packed by BentoML  and stored under the default directory
`~/bentoml/{service_name}/{service_version}` .
The BentoML file format contains all the code, files, and configs required to 
deploy the model for serving.


# 3. Containerize the model with docker

One common way of distributing this model API server for production deployment, is via
Docker containers. And BentoML provides a convenient way to do that.

Note that `docker` is __not available in Google Colab__. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a 
docker container serving the `IrisClassifier` prediction service created above:

In [9]:
!bentoml containerize IrisClassifier:latest -t iris-classifier:latest

[2021-02-14 21:37:35,172] INFO - Getting latest version IrisClassifier:20210214213445_DB9735
Found Bento: /home/theodore/bentoml/repository/IrisClassifier/20210214213445_DB9735
Containerizing IrisClassifier:20210214213445_DB9735 with local YataiService and docker daemon from local environment\WARNING: No swap limit support
|Build container image: iris-classifier:latest


Start a container with the docker image built in the previous step:

In [10]:
!docker run -p 5000:5000 iris-classifier:latest --workers=1 --enable-microbatch

[2021-02-14 20:39:08,593] INFO - Starting BentoML API server in production mode..
[2021-02-14 20:39:08,848] INFO - Running micro batch service on :5000
[2021-02-14 20:39:08 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2021-02-14 20:39:08 +0000] [10] [INFO] Starting gunicorn 20.0.4
[2021-02-14 20:39:08 +0000] [1] [INFO] Listening at: http://0.0.0.0:42851 (1)
[2021-02-14 20:39:08 +0000] [10] [INFO] Listening at: http://0.0.0.0:5000 (10)
[2021-02-14 20:39:08 +0000] [10] [INFO] Using worker: aiohttp.worker.GunicornWebWorker
[2021-02-14 20:39:08 +0000] [1] [INFO] Using worker: sync
[2021-02-14 20:39:08 +0000] [11] [INFO] Booting worker with pid: 11
[2021-02-14 20:39:08 +0000] [12] [INFO] Booting worker with pid: 12
[2021-02-14 20:39:08,923] INFO - Micro batch enabled for API `predict` max-latency: 10000 max-batch-size 2000
[2021-02-14 20:39:08,923] INFO - Your system nofile limit is 1048576, which means each instance of microbatch service is able to hold this number of connections at same ti

This made it possible to deploy BentoML bundled ML models with platforms such as
[Kubeflow](https://www.kubeflow.org/docs/components/serving/bentoml/),
[Knative](https://knative.dev/community/samples/serving/machinelearning-python-bentoml/),
[Kubernetes](https://docs.bentoml.org/en/latest/deployment/kubernetes.html), which
provides advanced model deployment features such as auto-scaling, A/B testing,
scale-to-zero, canary rollout and multi-armed bandit.

# Summary

This is a very short example how you can load a model from MLflow and serve it using BentoML. 

We recently looked into building an automated integration, the idea was to make BentoML support and serve the model format created in MLFlow directly. Although the team has concluded it is probably a really bad idea. The main difficulty of doing that is MLFlow's model format is not really designed for serving. And when turning a trained model to a prediction service, there are a number of things that may require the users' attention, which is not supported in MLFlow. In particular, what is the input/output data schema of the prediction endpoint, what are the local code dependencies, and how to preprocess a batch of input data, so it can take advantage of the micro-batching mechanism provided by BentoML, etc.

There might be other ways we can improve the integration with MLFlow, but for now, we decided to get started with this documentation on how users can potentially build a workflow that takes advantage of both frameworks.
